**Check the GPU**

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


**Install BLEURT**

In [ ]:
!git clone https://github.com/google-research/bleurt.git
!pip install -q ./bleurt

# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
# !unzip BLEURT-20.zip -d "/content/drive/MyDrive/Colab Notebooks/magistrska/models/bleurt-checkpoints"

# !rm BLEURT-20.zip

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (49/49), done.
  Preparing metadata (setup.py) ... done


**Install packages from pip**

In [ ]:
!pip install -q rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


**Import libraries**

In [ ]:
# from bleurt import score
# from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split

**Load data to DataFrame**

In [ ]:
import os
import pandas as pd

DIR_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/data"

def load_df_from_drive():
  return pd.read_csv(f"{DIR_PATH}/dataframe-kas.csv", encoding="utf-8")

In [ ]:
print("Loading data from drive...")
df = load_df_from_drive()
df = df.dropna()
df = df.reset_index(drop=True)
print("Finished.")

Loading data from drive...
Finished.


**Split into train, test and validation dataset (80 : 10 : 10)**

In [ ]:
train_df, test_and_validation_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=42)
test_df, validation_df = train_test_split(test_and_validation_df, test_size=0.5, shuffle=True, random_state=42)
train_df.shape, test_df.shape, validation_df.shape

del df

**Draw a plot of validation loss change for a model**

In [4]:
import matplotlib.pyplot as plt
import pandas as pd

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"
testing_filename_csv = f"{RESULTS_PATH}/google/long-t5-tglobal-base/to_eng/percentage_40.0/validation_results_epoch=19-validation_loss=2.58904.csv"
results = pd.read_csv(testing_filename_csv)

loss_values = list(results.get("loss"))
epochs = range(0, len(loss_values))

# Find the epoch with the minimum loss value
min_loss = min(loss_values)
min_epoch = loss_values.index(min_loss)

# Create the plot
plt.figure(figsize=(10, 6))

# Plot the loss values
plt.plot(epochs, loss_values, label='Validacijska izguba', color='blue', linewidth=1)

# Mark the minimum loss value
plt.scatter(min_epoch, min_loss, color='red', zorder=5)
plt.text(min_epoch, min_loss, f'  Min. izguba\n  {min_loss:.3f}', fontsize=12, verticalalignment='bottom', color='red')

# Add title and labels
plt.title(f"Sprememba izgube na validacijski množici", fontsize=16)
plt.xlabel('Iteracija učenja', fontsize=14)
plt.ylabel('Izguba', fontsize=14)

# Add a legend
plt.legend(loc='upper right', fontsize=12)

# Add grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Save the plot to a file
plt.savefig(f"{RESULTS_PATH}/google/long-t5-tglobal-base/to_eng/percentage_40.0/validation_loss_plot_with_min.png")

# Show the plot
plt.show()

TypeError: 'NoneType' object is not iterable

**Export 30 random examples to a JSON file**

In [1]:
import json
import pandas as pd

random_examples = [6, 12, 47, 631, 56, 134, 219, 326, 403, 639, 183, 272,
                   118, 649, 313, 497, 231, 352, 278, 528, 185, 97, 70, 688,
                   332, 263, 408, 644, 31, 544]

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"
LONGT5_PATH = f"{RESULTS_PATH}/google/long-t5-tglobal-base/to_eng/percentage_100.0/testing_results_epoch=14-validation_loss=2.93539.csv"
PEGASUS_X_PATH = f"{RESULTS_PATH}/google/pegasus-x-base/to_eng/percentage_100.0/testing_results_epoch=16-validation_loss=3.46449.csv"
BIG_BIRD_PATH = f"{RESULTS_PATH}/google/bigbird-pegasus-large-arxiv/to_eng/percentage_100.0/testing_results_epoch=16-validation_loss=4.36044.csv"

longt5_results = pd.read_csv(LONGT5_PATH)
pegasus_x_results = pd.read_csv(PEGASUS_X_PATH)
big_bird_results = pd.read_csv(BIG_BIRD_PATH)

all_data = {}
for example_id in random_examples:
  example_data = {}
  longt5_row = longt5_results.iloc[example_id]
  pegasus_x_row = pegasus_x_results.iloc[example_id]
  big_bird_row = big_bird_results.iloc[example_id]

  example_data["reference_summary_eng"] = longt5_row["abstract_eng"].replace("\n", "")
  example_data["longt5_generated_summary_eng"] = longt5_row["generated_summary_eng"]
  example_data["pegasus_x_generated_summary_eng"] = pegasus_x_row["generated_summary_eng"]
  example_data["big_bird_generated_summary_eng"] = big_bird_row["generated_summary_eng"]

  all_data[example_id] = example_data

json_data = json.dumps(all_data, indent=4, ensure_ascii=False)

with open(f"{RESULTS_PATH}/analysis-ready.json", "w", encoding='utf-8') as outfile:
  outfile.write(json_data)


**Create 3 text files (first three from above) for ChatGPT summary generation**

In [ ]:
RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"

random_examples = [6, 12, 47]
for example_id in random_examples:
  example = test_df.iloc[example_id]
  example_text = example["text"]

  with open(f"{RESULTS_PATH}/text_{example_id}.txt", "w", encoding='utf-8') as text_file:
    print(f"Generating text_{example_id}.txt in {RESULTS_PATH}...")
    text_file.write(example_text)
    print(f"Done\n")


Generating text_6.txt in /content/drive/MyDrive/Colab Notebooks/magistrska/results...
Done

Generating text_12.txt in /content/drive/MyDrive/Colab Notebooks/magistrska/results...
Done

Generating text_47.txt in /content/drive/MyDrive/Colab Notebooks/magistrska/results...
Done

